source API URL : "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

JSON Target File Path : "abfss://bronze@datalakestorageaccountname.dfs.core.windows.net/geo-location/"

In [0]:
geoLocationSourceAPIURL = "https://geocoding-api.open-meteo.com/v1/search?name=kovilpatti&count=10&language=en&format=json"

geoLocationSourceAPIBaseURL = "https://geocoding-api.open-meteo.com/v1/search?name="
geoLocationSourceAPIURLOptions = "&count=10&language=en&format=json"

geoLocationSinkLayerName = 'bronze'
geoLocationSinkStorageAccountName = 'lckudadatalakehousedev'
geoLocationSinkFolderName = 'geo-location'

geoLocationSinkFolderPath = f"abfss://{geoLocationSinkLayerName}@{geoLocationSinkStorageAccountName}.dfs.core.windows.net/{geoLocationSinkFolderName}"

In [0]:
import requests
import json
import pandas as pds

In [0]:
geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL).json()
geoLocationPandasDF = pds.DataFrame(geoLocationAPIResponse)
geoLocationSparkDF = spark.createDataFrame(geoLocationPandasDF)

In [0]:
dailyPricingMarketNamesDF = spark.sql("SELECT MARKET_NAME from pricing_analytics.gold.reporting_dim_market_gold ")

In [0]:
marketNames = [row["MARKET_NAME"] for row in dailyPricingMarketNamesDF.collect()]
print(f"Fetched {len(marketNames)} market names.")

geoLocationAPIResponseList = []
for marketName in marketNames:
    geoLocationSourceAPIURL = f"{geoLocationSourceAPIBaseURL}{marketName}{geoLocationSourceAPIURLOptions}"
    print(f"Requesting GeoLocation API for market: {marketName}")
    geoLocationAPIResponse = requests.get(geoLocationSourceAPIURL).json()
    if isinstance(geoLocationAPIResponse, dict):
        geoLocationAPIResponseList.append(geoLocationAPIResponse)
        print(f"Received valid response for market: {marketName}")
    else:
        print(f"Invalid response for market: {marketName}")

print(f"Total valid API responses: {len(geoLocationAPIResponseList)}")

geoLocationSparkDF = spark.createDataFrame(pds.DataFrame(geoLocationAPIResponseList))

(geoLocationSparkDF
 .filter("results.admin1 IS NOT NULL")
 .write
 .mode("overwrite")
 .json(geoLocationSinkFolderPath))

print(f"GeoLocation data written to {geoLocationSinkFolderPath}")

In [0]:
from pyspark.sql.functions import col, array_contains
geoLocationBronzeDF = (spark
                       .read
                       .json(geoLocationSinkFolderPath))
                       

display(geoLocationBronzeDF)